In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model as lm

In [2]:
buildings = pd.read_csv('Houses_Sold.csv')
buildings.dropna(axis=0, inplace=True)

In [3]:
bb_freq = buildings.loc[(buildings['house_size'] <= 5000),].groupby(['bed','bath']).size().reset_index(name='count')
bb_freq = bb_freq.loc[(bb_freq['bed']) > (bb_freq['bath'])]
max_beds = np.min(bb_freq.loc[(bb_freq['count'] == 1),]['bed'])-1
print(f'maximum bedrooms are {max_beds}')
# max_baths = np.max(bb_freq['bath'])
# print(f'maximum bathrooms are {max_baths}')

maximum bedrooms are 12.0


In [4]:
houses = buildings.loc[ (buildings['bed'] <= max_beds) & (buildings['bed'] > buildings['bath']) & (buildings['house_size'] <= 5000) & (buildings['price'] > 1000), ]
# houses = buildings.loc[(buildings['bed'] < 7) & (buildings['bath'] < buildings['bed']),]
houses_ar = np.array(houses.select_dtypes(include=np.number).iloc[:,1:])

PCA to reduce dimensions of housing dataframe

In [ ]:
h_scaled = StandardScaler().fit_transform(houses_ar)
pca = PCA(n_components=2)
pca.fit(h_scaled)
h_PCs = pca.fit_transform(h_scaled)

In [ ]:
mask = h_PCs[:,0] < 10
plot_ar = h_PCs[mask]

fig, ax = plt.subplots()
ax.scatter(h_PCs[:,0], h_PCs[:,1])

In [ ]:
fig, ax = plt.subplots()
ax.scatter(houses['house_size'], np.log(houses['price']))
ax.set_ylabel('ln(Price)')
ax.set_xlabel('House Square Footage')

In [ ]:
onehot_state = pd.get_dummies(houses['state'])
scale = StandardScaler()
scale.fit(np.array(houses[['house_size', 'acre_lot']]))
X = np.array(houses[['house_size', 'acre_lot']])
X = np.column_stack((X, onehot_state))
scale.fit(np.array(houses['price']).reshape(-1,1))
y = np.array(houses['price']).reshape(-1,1)

gamma = lm.GammaRegressor()
gamma.fit(X,y)
# gamma.score(X,y)

In [ ]:
gamma.coef_

In [ ]:
y_hat = gamma.predict(X)
mask = houses['state']=='California'
fig, ax = plt.subplots()
ax.scatter(houses.loc[mask,'house_size'], houses.loc[mask,'price'])
order = np.argsort(houses.loc[mask, 'house_size'])
ax.plot(houses.loc[mask,'house_size'].iloc[order], y_hat[mask][order], color='red', linewidth=2)
ax.set_ylim(0.001e7, 1.5e7)
plt.show()

In [5]:
disaster = pd.read_csv('NRI_Table_Counties.csv')
disaster['NRI_ID'] = disaster['NRI_ID'].astype(str).str.replace(r'^C', '', regex=True)
disaster['NRI_ID'] = pd.to_numeric(disaster['NRI_ID'], errors='coerce').astype('Int64')  # nullable integer dtype

check = disaster.loc[disaster['COUNTY'] == 'Fulton', :]

In [7]:
ZIPS = pd.read_csv('ZIP_COUNTY_062025.csv')

In [ ]:
dis_zip_county = disaster.merge(ZIPS[['ZIP', 'COUNTY', 'USPS_ZIP_PREF_CITY']], how='left', left_on='NRI_ID', right_on='COUNTY')
# There are 11 records (all in Conneticut and American Somoa) that do not have county information after the merge
check = dis_zip_county.loc[dis_zip_county['ZIP'].isna(),:]

In [12]:
Complete_df = houses.merge(dis_zip_county, how='left', left_on='zip_code', right_on='ZIP')
check = Complete_df.loc[Complete_df['ZIP'].isna(),:]

In [13]:
Complete_df.shape[0]-check.shape[0]

999547